In [ ]:
import pandas as pd
import glob
import tensorflow as tf

In [ ]:
train = pd.read_csv(r"../input/petfinder-pawpularity-score/train.csv")
test = pd.read_csv(r"../input/petfinder-pawpularity-score/test.csv")

In [ ]:
train = train.sort_values(by='Id', ascending=True)
test = test.sort_values(by='Id', ascending=True)

In [ ]:
train_images = glob.glob("../input/petfinder-pawpularity-score/train/*")
test_images = glob.glob("../input/petfinder-pawpularity-score/test/*")

In [ ]:
train['Image'] = train_images
test['Image'] = test_images

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_dataset = train_generator.flow_from_dataframe(dataframe=train,
                                                        x_col="Image",
                                                        y_col="Pawpularity",
                                                        subset="training",
                                                        target_size=(256,256),
                                                        shuffle=True,
                                                        class_mode="raw",
                                                        validate_filenames=False)

test_dataset = test_generator.flow_from_dataframe(dataframe=test,
                                                x_col="Image",
                                                y_col=None,
                                                target_size=(256,256),
                                                shuffle=False,
                                                class_mode=None,
                                                validate_filenames=False)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), input_shape=(256, 256, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(filters=60, kernel_size=(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=tf.keras.metrics.RootMeanSquaredError())
model.fit(x=train_dataset, epochs=15)
model.evaluate(test_dataset)

In [ ]:
prediction = model.predict(test_dataset)
prediction_list = []
for i in prediction:
    prediction_list.append(float(i))

In [ ]:
test_predict = pd.DataFrame({'Id':test['Id'], 'Pawpularity':prediction_list})
test_predict.to_csv("submission.csv", index = False)